# Video: Flattening Arrays with Views

This video covers the different choices and tradeoffs flattening multi-dimensional arrays back down to one dimension.




In [32]:
import numpy as np

In [33]:
x_contiguous = np.array([[0, 1, 2], [3, 4, 5]])
x_contiguous

array([[0, 1, 2],
       [3, 4, 5]])

In [34]:
x_contiguous.flags

  C_CONTIGUOUS : True
  F_CONTIGUOUS : False
  OWNDATA : True
  WRITEABLE : True
  ALIGNED : True
  WRITEBACKIFCOPY : False

In [35]:
def check(a):
    print("ID", id(a))
    print("DATA")
    print(a)
    if a.base is not None:
        print("BASE")
        print(a.base)
        print("BASE ID", id(a.base))
    print("C_CONTIGUOUS", a.flags["C_CONTIGUOUS"])
    print("OWNDATA", a.flags["OWNDATA"])
    print("STRIDES", a.strides)

check(x_contiguous)

ID 140020203873904
DATA
[[0 1 2]
 [3 4 5]]
C_CONTIGUOUS True
OWNDATA True
STRIDES (24, 8)


In [36]:
check(np.reshape(x_contiguous, -1))

ID 140020203872080
DATA
[0 1 2 3 4 5]
BASE
[[0 1 2]
 [3 4 5]]
BASE ID 140020203873904
C_CONTIGUOUS True
OWNDATA False
STRIDES (8,)


In [37]:
check(np.ravel(x_contiguous))

ID 140020168542032
DATA
[0 1 2 3 4 5]
BASE
[[0 1 2]
 [3 4 5]]
BASE ID 140020203873904
C_CONTIGUOUS True
OWNDATA False
STRIDES (8,)


In [38]:
x_not_contiguous = x_contiguous[:,::2]
check(x_not_contiguous)

ID 140020168543664
DATA
[[0 2]
 [3 5]]
BASE
[[0 1 2]
 [3 4 5]]
BASE ID 140020203873904
C_CONTIGUOUS False
OWNDATA False
STRIDES (24, 16)


In [39]:
check(np.reshape(x_not_contiguous, -1))

ID 140020168545488
DATA
[0 2 3 5]
BASE
[[0 2]
 [3 5]]
BASE ID 140020203872080
C_CONTIGUOUS True
OWNDATA False
STRIDES (8,)


In [40]:
mystery_base = np.reshape(x_not_contiguous, -1).base
check(mystery_base)

ID 140020203872080
DATA
[[0 2]
 [3 5]]
C_CONTIGUOUS True
OWNDATA True
STRIDES (16, 8)


In [41]:
check(np.ravel(x_not_contiguous))

ID 140020168545008
DATA
[0 2 3 5]
C_CONTIGUOUS True
OWNDATA True
STRIDES (8,)


* As expected, ravel returns a new array in this case.
* Both because a view would not work, and because a view would not be contiguous, but ravel promises a contiguous array.
* Wrapping up, why did I spend so much time looking into when these functions used views, and when they did not?
* First, creating a view is very fast. Operations that are just views are basically instant. If you need to copy data, that will take a while if you have a lot of data.
* Second, copying is slow, but usually you get contiguous arrays afterwards, and those will be faster to operate on when you get to doing the real calculations. Not to say that you want a lot of copying, but one copy at the end of a view transformations might speed up the rest of your code.
* Third, it is good to know which behavior is happening. Both for speed, and because if you have a view, there will be data sharing. You really want to know if writing to one array changes another. It is always a bad surprise to find that someone, including yourself, has been overwriting your data unexpectedly.
* This video has been about understanding views to flatten arrays and checking or confirming that they were used.
* There is one more function, actually a method, to flatten arrays.
* Every NumPy array has a method flatten that will return a copy of the array flattened into one dimension.

In [42]:
check(x_not_contiguous.flatten())

ID 140020168545008
DATA
[0 2 3 5]
C_CONTIGUOUS True
OWNDATA True
STRIDES (8,)


**Code Notes:**
* The NumPy function [`numpy.reshape`](https://numpy.org/doc/stable/reference/generated/numpy.reshape.html), referenced as `np.reshape`, returns a new array with the same data and a specified shape.
  * `reshape` returns a view whenever possible.
  * If you specify length -1 for one axis in the shape, NumPy will calculate that size for you based on the other sizes and the number of data elements.
* The NumPy function [`numpy.ravel`](https://numpy.org/doc/stable/reference/generated/numpy.ravel.html) returns a 1-dimensional array with the same contents as the input, and guarantees that the array will be contiguous.
  * If possible, `ravel` will return a view.
* The NumPy ndarray method [`numpy.ndarray.flatten`](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.flatten.html) returns a copy of the array's data as a one-dimensional array.
  * `flatten` never attempts to return a view which distinguishes it from `numpy.reshape` and `numpy.ravel`.